# TensorFlow 패키지 소개

TensorFlow 패키지는 Theano 패키지와 마찬가지로 선형 대수 심볼 컴파일러(Symbolic Linear Algebra Compiler)이다. Theano에 비해 분산처리 기능이 강화되어 있다.

## TensorFlow 기본 사용법

Theano와 유사하게 TensorFlow도 다음과 같은 과정을 거쳐 사용한다.

1. 심볼 변수 정의
2. 심볼 관계 정의
3. 세션 정의
4. 세션 사용

세션(Session)은 Theano의 함수와 유사한 역할을 하며 실제 심볼 변수의 관계를 분석하고 값을 계산해준다.

## 심볼 변수 정의

TensorFlow는 `Variable` 명령으로 정의하며 Theano와 달리 각 심볼 변수의 형태를 스칼라, 벡터, 행렬 등으로 명시적으로 정의하지 않는 대신 초기값을 이용해서 형태를 지정한다.

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [4]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

## 심볼 관계 정의

이미 만들어진 심볼 변수에 일반 사칙연산이나 TensorFlow 수학 함수를 사용하여 종속 변수 역할을 하는 심볼 변수를 만들 수 있다.

In [5]:
z = x + y

In [6]:
type(z)

tensorflow.python.framework.ops.Tensor

In [7]:
u = tf.log(z)

In [8]:
type(u)

tensorflow.python.framework.ops.Tensor

## 세션

TensorFlow의 세션은 Theano의 함수와 유사한 역할을 하며 실제로 계산 그래프를 생성하고 값을 계산하기위한 환경을 제공한다. Theano의 함수와 달리 세션 생성과 실행 시작, 종료를 명시해야 한다.

* 세션 생성: `Session` 객체 생성
* 세션 사용: `run` 메서드
* 세션 종료: `close` 메서드. with 문으로 대체하기도 한다.

In [9]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    print(sess.run(z))    
    print(sess.run(u))

3.0
1.09861


## 미분

TensorFlow 도 심볼릭 연산에 의한 미분 계산이 가능하다.

In [10]:
f = x ** 2
fx = tf.gradients(f, [x])

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    print(sess.run(f))    
    print(sess.run(fx))

1.0
[2.0]


## 최적화

TensorFlow에는 Theano와 달리 최적화를 위한 `GradientDescentOptimizer` 등의 클래스가 미리 구현되어 있으므로 사용자가 구현할 필요가 없다.

## 선형 회귀

In [11]:
# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but Tensorflow will
# figure that out for us.)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(loss), sess.run(W), sess.run(b))

# Learns best fit is W: [0.1], b: [0.3]
sess.close()

0 0.00950747 [ 0.31143823] [ 0.25803351]
20 0.000171959 [ 0.14356297] [ 0.27430949]
40 1.06265e-05 [ 0.11082928] [ 0.29361364]
60 6.56683e-07 [ 0.10269205] [ 0.29841241]
80 4.05793e-08 [ 0.10066921] [ 0.29960537]
100 2.50735e-09 [ 0.10016634] [ 0.2999019]
120 1.54874e-10 [ 0.10004134] [ 0.29997563]
140 9.55588e-12 [ 0.10001028] [ 0.29999396]
160 5.99956e-13 [ 0.10000258] [ 0.29999849]
180 3.90532e-14 [ 0.10000066] [ 0.29999962]
200 2.12275e-15 [ 0.10000015] [ 0.29999992]


## MNIST 예제

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

import argparse
tf.app.flags.FLAGS = tf.python.platform.flags._FlagValues()
tf.app.flags._global_parser = argparse.ArgumentParser()

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data for unit testing.')
flags.DEFINE_integer('max_steps', 200, 'Number of steps to run trainer.')
flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.')
flags.DEFINE_float('dropout', 0.9, 'Keep probability for training dropout.')
flags.DEFINE_string('data_dir', '/home/dockeruser/data', 'Directory for storing data')
flags.DEFINE_string('summaries_dir', '/home/dockeruser/logs', 'Summaries directory')


def train():
    # Import data
    mnist = input_data.read_data_sets(FLAGS.data_dir,
                                      one_hot=True,
                                      fake_data=FLAGS.fake_data)

    sess = tf.InteractiveSession()

    # Create a multilayer model.

    # Input placehoolders
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.image_summary('input', image_shaped_input, 10)

    # We can't initialize these variables to 0 - the network will get stuck.
    def weight_variable(shape):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def variable_summaries(var, name):
        """Attach a lot of summaries to a Tensor."""
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.scalar_summary('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.scalar_summary('sttdev/' + name, stddev)
        tf.scalar_summary('max/' + name, tf.reduce_max(var))
        tf.scalar_summary('min/' + name, tf.reduce_min(var))
        tf.histogram_summary(name, var)

    def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
        """Reusable code for making a simple neural net layer.
        It does a matrix multiply, bias add, and then uses relu to nonlinearize.
        It also sets up name scoping so that the resultant graph is easy to read,
        and adds a number of summary ops.
        """
        # Adding a name scope ensures logical grouping of the layers in the graph.
        with tf.name_scope(layer_name):
            # This Variable will hold the state of the weights for the layer
            with tf.name_scope('weights'):
                weights = weight_variable([input_dim, output_dim])
                variable_summaries(weights, layer_name + '/weights')
            with tf.name_scope('biases'):
                biases = bias_variable([output_dim])
                variable_summaries(biases, layer_name + '/biases')
            with tf.name_scope('Wx_plus_b'):
                preactivate = tf.matmul(input_tensor, weights) + biases
                tf.histogram_summary(layer_name + '/pre_activations', preactivate)
            activations = act(preactivate, 'activation')
            tf.histogram_summary(layer_name + '/activations', activations)
            return activations

    hidden1 = nn_layer(x, 784, 500, 'layer1')

    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        tf.scalar_summary('dropout_keep_probability', keep_prob)
        dropped = tf.nn.dropout(hidden1, keep_prob)

    y = nn_layer(dropped, 500, 10, 'layer2', act=tf.nn.softmax)

    with tf.name_scope('cross_entropy'):
        diff = y_ * tf.log(y)
        with tf.name_scope('total'):
            cross_entropy = -tf.reduce_mean(diff)
        tf.scalar_summary('cross entropy', cross_entropy)

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.scalar_summary('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.merge_all_summaries()
    train_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/train', sess.graph)
    test_writer = tf.train.SummaryWriter(FLAGS.summaries_dir + '/test')
    tf.initialize_all_variables().run()

    # Train the model, and also write summaries.
    # Every 10th step, measure test-set accuracy, and write test summaries
    # All other steps, run train_step on training data, & add training summaries

    def feed_dict(train):
        """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
        if train or FLAGS.fake_data:
            xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
            k = FLAGS.dropout
        else:
            xs, ys = mnist.test.images, mnist.test.labels
            k = 1.0
        return {x: xs, y_: ys, keep_prob: k}

    for i in range(FLAGS.max_steps):
        if i % 10 == 0:  # Record summaries and test-set accuracy
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
            test_writer.add_summary(summary, i)
            print('Accuracy at step %s: %s' % (i, acc))
        else:  # Record train set summaries, and train
            if i % 100 == 99:  # Record execution stats
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                      feed_dict=feed_dict(True),
                                      options=run_options,
                                      run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
                train_writer.add_summary(summary, i)


def main(_):
    if tf.gfile.Exists(FLAGS.summaries_dir):
        tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
    tf.gfile.MakeDirs(FLAGS.summaries_dir)
    train()
    
    
main("")

Extracting /home/dockeruser/data/train-images-idx3-ubyte.gz
Extracting /home/dockeruser/data/train-labels-idx1-ubyte.gz
Extracting /home/dockeruser/data/t10k-images-idx3-ubyte.gz
Extracting /home/dockeruser/data/t10k-labels-idx1-ubyte.gz
Accuracy at step 0: 0.0913
Accuracy at step 10: 0.6582
Accuracy at step 20: 0.8026
Accuracy at step 30: 0.8452
Accuracy at step 40: 0.8714
Accuracy at step 50: 0.8778
Accuracy at step 60: 0.8815
Accuracy at step 70: 0.882
Accuracy at step 80: 0.8814
Accuracy at step 90: 0.893
Adding run metadata for 99
Accuracy at step 100: 0.8994
Accuracy at step 110: 0.916
Accuracy at step 120: 0.9154
Accuracy at step 130: 0.9155
Accuracy at step 140: 0.9225
Accuracy at step 150: 0.9185
Accuracy at step 160: 0.9249
Accuracy at step 170: 0.9233
Accuracy at step 180: 0.9202
Accuracy at step 190: 0.9233
Adding run metadata for 199


## TensorBoard

TensorFlow는 theano와 달리 모형 내부와 결과를 모니터링 할 수 있는 TensorBoard 라는 웹서버를 제공한다.

TensorBoard 웹서버는 포트 6006을 사용하므로 만약 도커를 사용하는 경우에는 다음과 같이 포트를 열고 실행해야 한다.

```
docker run --name rpython -it -p 8888:8888 -p 6006:6006 datascienceschool/rpython
```

TensorBoard를 가동하기 위해서는 콘솔에서 다음과 같이 로그 디렉토리를 설정하여 실행한다.

```
$ tensorboard --logdir=/home/dockeruser/logs &
```

위와 같이 실행한 다음에는 다음 주소로 연결하면 TensorBoard 화면을 볼 수 있다.

http://192.168.99.100:6006


<img src="https://datascienceschool.net/upfiles/cb3cc08fcafd437e873ca710a23d7c0a.png">

TensorFlow는 Theano처럼 `theano.printing.pydotprint` 명령을 지원하지 않는 대신 TensorBoard를 통해 그래프를 보여줄 수 있다.